In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy

In [3]:
movies = pd.read_csv('movie_archive\movies_metadata.csv')
ratings_small = pd.read_csv('movie_archive\\ratings_small.csv')
links_small = pd.read_csv('movie_archive\links_small.csv')

C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), stop_words='english')
%time tfidf_matrix = tfidf.fit_transform(movies['title'].apply(str) + " " + movies['overview'].apply(str))
tfidf_matrix

Wall time: 15 s


<45466x2407186 sparse matrix of type '<class 'numpy.float64'>'
	with 3943994 stored elements in Compressed Sparse Row format>

In [5]:
userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))

# get all movie vector rated from user
def get_user_rated_movies(ids):
   
    movie_vector_list = []
    for i in ids:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
        try:
            index = movieid_list.index(int(tmdbId))
            movie_vector = tfidf_matrix[index]
            movie_vector_list.append(movie_vector)
        except ValueError:
            print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies
        
    movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors

# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}

def create_user_vectors():
    for uid in userid_list:
        user_profile = ratings_small.loc[ratings_small['userId'] == uid]
        user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
        try:
            user_vector = user_rated_movies_vector_list[0]
            for x in user_rated_movies_vector_list[1:]:
                user_vector = user_vector + x

            user_norm = sklearn.preprocessing.normalize(user_vector)
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished")
        except TypeError:
            print(str(uid) + " didnt work, coo matrix parse error")

%time create_user_vectors()


1 finished
ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
2 finished
3 finished
4 finished
5 finished
6 finished
ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
ValueError tmdbId: 5173    191903.0
Name: tmdbId, dtype: float64
13 finished
14 finished
ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
ValueError tmdbId: 5826   NaN
Name: tmdbId, dtype: float64
ValueError tmdbId: 7923    58423.0
Name: tmdbId, dtype: float64
15 finished
16 finished
17 finished
18 finished
19 finished
ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
20 finished
21 finished
22 finished
23 finished
24 finished
25 finished
ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
26 finished
27 finished
28 finished
29 finished
ValueError tmdbId: 2282   NaN
Name: tmdbId, dtype: float64
30 finished
31 finished
32 finished
33 finished
34 finished
35 finished
ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: fl

In [6]:
recommendation_users = {}
def sort_user_vectors():
    for uid in userid_list:
        user_vec = all_user_profiles[uid]
        cosine_similarity_user_movies = cosine_similarity(user_vec, tfidf_matrix)
        top_similar = cosine_similarity_user_movies[0].argsort()[:100:-1]
        recommendation_users[uid] = [(cosine_similarity_user_movies[0][i], movies['id'][i]) for i in top_similar]

%time sort_user_vectors()

Wall time: 6min 49s


In [7]:
%%time

percentage_already_seen = []
for uid in userid_list:
    user_profile = ratings_small.loc[ratings_small['userId'] == uid]
    user_recommendation = recommendation_users[uid]

    user_profile_mid_list = user_profile['movieId'].to_list()
    user_tmdb_list = []
    for m in user_profile_mid_list:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
        try:
            user_tmdb_list.append(int(tmdbId))
        except ValueError:
            print("ValueError: " + str(tmdbId))
    
    m_count = 0
    for movie_cosine, mid in user_recommendation[:len(user_profile)]:
        try:
            if int(mid) in user_tmdb_list:
                m_count += 1
        except ValueError:
            print("ValueError: " + str(mid) + " not in tmdbid list")    

    if m_count == len(user_profile):
        print(str(uid) + " : the first movies are already viewed")
    else:
        print(str(m_count) + " of " + str(len(user_profile)) + " in first recommended already seen")

    p = m_count/len(user_profile)
    percentage_already_seen.append(p)

average_percentage = sum(percentage_already_seen) / len(percentage_already_seen)
print("Average Percentage of already seen movies: " + str(average_percentage))


1 : the first movies are already viewed
ValueError: 607   NaN
Name: tmdbId, dtype: float64
75 of 76 in first recommended already seen
3 : the first movies are already viewed
4 : the first movies are already viewed
5 : the first movies are already viewed
6 : the first movies are already viewed
ValueError: 607   NaN
Name: tmdbId, dtype: float64
87 of 88 in first recommended already seen
8 : the first movies are already viewed
9 : the first movies are already viewed
10 : the first movies are already viewed
35 of 38 in first recommended already seen
12 : the first movies are already viewed
52 of 53 in first recommended already seen
14 : the first movies are already viewed
ValueError: 607   NaN
Name: tmdbId, dtype: float64
ValueError: 5826   NaN
Name: tmdbId, dtype: float64
439 of 1700 in first recommended already seen
16 : the first movies are already viewed
359 of 363 in first recommended already seen
18 : the first movies are already viewed
409 of 423 in first recommended already seen
Va